In [168]:
import cv2 as cv
import numpy as np

In [169]:
img = cv.imread("example.jpg")

In [170]:
cv.imshow("img", img)
cv.waitKey() 
cv.destroyAllWindows()

In [171]:
print(img[1151,686]) #print pixel's color

[176 175 177]


In [172]:
print(img[1151,686, 0]) #sicne it's BGR, it prints the Blue value of pixel at 1141,686

176


In [173]:
img[100,100] = (0,0,255) #changing red value at pixel 100,100
print(img[100,100,2])

255


In [174]:
print(img.shape)

(1257, 1993, 3)


In [175]:
print(img.size) #1257*1993*3

7515603


In [176]:
img.size

7515603

In [177]:
img = cv.imread("pencils.jpg")

x1,y1 = 100,960
x2,y2 = 200,960

piece = img[x1:y1, x2:y2]
img[x1:y1, x2+500:y2+500] = piece

cv.imshow("img", img)
cv.waitKey() 
cv.destroyAllWindows()

In [178]:
#Image Addition
img1 = cv.imread("Future1280x720.jpg")
img2 = cv.imread("Background1280x720.jpg")

img3 = cv.add(img1, img2)
img4 = cv.addWeighted(img1, 0.6, img2, 0.4,0)

cv.imshow("Image addition", img3)
cv.imshow("Image addition wighted", img4)
cv.waitKey()
cv.destroyAllWindows()

In [179]:
#blending images
img1 = cv.imread("Future1280x720.jpg")
img2 = cv.imread("Background1280x720.jpg")

rows,columns,channels = img1.shape
roi = img2[0:rows, 0:columns]

img1gray = cv.cvtColor(img1,cv.COLOR_BGR2GRAY)
cv.imshow("img1gray", img1gray)

ret, img1mask = cv.threshold(img1gray, 80,255, cv.THRESH_BINARY)
cv.imshow("img1mask", img1mask)

img1invert = cv.bitwise_not(img1mask)
cv.imshow("img1invert", img1invert)

img2_bg_invert = cv.bitwise_and(roi,roi,mask = img1invert)
cv.imshow("img2_bg_invert", img2_bg_invert)

img2_bg_mask = cv.bitwise_and(roi,roi,mask = img1mask)
cv.imshow("img2_bg_mask", img2_bg_mask)

cv.waitKey()
cv.destroyAllWindows()

In [180]:
#Colorspaces
import cv2 as cv
flags = [i for i in dir(cv) if i.startswith('COLOR_')]
print(flags)

['COLOR_BAYER_BG2BGR', 'COLOR_BAYER_BG2BGRA', 'COLOR_BAYER_BG2BGR_EA', 'COLOR_BAYER_BG2BGR_VNG', 'COLOR_BAYER_BG2GRAY', 'COLOR_BAYER_BG2RGB', 'COLOR_BAYER_BG2RGBA', 'COLOR_BAYER_BG2RGB_EA', 'COLOR_BAYER_BG2RGB_VNG', 'COLOR_BAYER_GB2BGR', 'COLOR_BAYER_GB2BGRA', 'COLOR_BAYER_GB2BGR_EA', 'COLOR_BAYER_GB2BGR_VNG', 'COLOR_BAYER_GB2GRAY', 'COLOR_BAYER_GB2RGB', 'COLOR_BAYER_GB2RGBA', 'COLOR_BAYER_GB2RGB_EA', 'COLOR_BAYER_GB2RGB_VNG', 'COLOR_BAYER_GR2BGR', 'COLOR_BAYER_GR2BGRA', 'COLOR_BAYER_GR2BGR_EA', 'COLOR_BAYER_GR2BGR_VNG', 'COLOR_BAYER_GR2GRAY', 'COLOR_BAYER_GR2RGB', 'COLOR_BAYER_GR2RGBA', 'COLOR_BAYER_GR2RGB_EA', 'COLOR_BAYER_GR2RGB_VNG', 'COLOR_BAYER_RG2BGR', 'COLOR_BAYER_RG2BGRA', 'COLOR_BAYER_RG2BGR_EA', 'COLOR_BAYER_RG2BGR_VNG', 'COLOR_BAYER_RG2GRAY', 'COLOR_BAYER_RG2RGB', 'COLOR_BAYER_RG2RGBA', 'COLOR_BAYER_RG2RGB_EA', 'COLOR_BAYER_RG2RGB_VNG', 'COLOR_BGR2BGR555', 'COLOR_BGR2BGR565', 'COLOR_BGR2BGRA', 'COLOR_BGR2GRAY', 'COLOR_BGR2HLS', 'COLOR_BGR2HLS_FULL', 'COLOR_BGR2HSV', 'COLOR_

In [181]:
import cv2 as cv
import numpy as np

cap = cv.VideoCapture(0)
while True:
    _,bgr_frame = cap.read()
    
    hsv_frame = cv.cvtColor(bgr_frame, cv.COLOR_BGR2HSV)
    
    lowerb = np.array([0,0,0]) 
    upperb = np.array([100,184,240]) #near to my skin color (it depends a lot from light condition)
    
    mask = cv.inRange(hsv_frame, lowerb, upperb, cv.THRESH_BINARY)
    #mask = cv.inRange(bgr_frame, lowerb, upperb, cv.THRESH_BINARY) 
    
    #res = cv.bitwise_and(bgr_frame,bgr_frame, mask = mask)
    res = cv.bitwise_not(hsv_frame,hsv_frame, mask = mask)
    
    res2bgr = cv.cvtColor(res, cv.COLOR_HSV2BGR)
    
    cv.imshow("bgr_frame",bgr_frame)
    cv.imshow("hsv_frame",hsv_frame)
    cv.imshow("res",res)
    cv.imshow("res2bgr",res2bgr)
    if cv.waitKey(1) == 113:
        break
        
cap.release()
cv.destroyAllWindows()

## Thresholding
As said in the previous cell, global thresholding is not good when working with different light condition.  
To avoid this problems, we can suse Adaptive Thresholding.  
In few words, adaptive thresholding allows us to have different thresholding values for different areas.  
It has 3 input paramethers:  
1. Adaptive method:  
   - ADAPTIVE_THRESH_MEAN_C: The threshold value is the mean of neighbourhood area  
   - ""_""_GAUSSIAN_C: threshold value is the wheighted sum of neighbourhood values  
            
            
2. Block Size: How big are the neighbourhood areas   
   
3. Constant subtracted from the mean calculated  

In [182]:
import cv2 as cv
import numpy as np

cap = cv.VideoCapture(0) #capture

while True:
    _,frame = cap.read() #reading frames
    
    grayFrame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY) #from bgr to black and gray shadows
    blurredFrame = cv.GaussianBlur(grayFrame, (5,5), 7) #blurring to remove image noise
                                        #src,      maxValue  #adaptive threshold method,     threshold type, block dim to calculate means, C
    thresholdFrame = cv.adaptiveThreshold(blurredFrame, 200, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 19, 3)
    ret, thresholdFrame = cv.threshold(thresholdFrame, 25, 255, cv.THRESH_BINARY) #used to clarify the result
    
    cv.imshow("AdaptiveThreshold", thresholdFrame)
    if cv.waitKey(1) == 113:
        break
        
cap.release()
cv.destroyAllWindows()